In [80]:
## Libraries

import sys
sys.path.append('/home/jupyter/DS_O2_Tools')
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.cloud import storage
import yaml
import pickle
import decimal

import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from matplotlib.ticker import PercentFormatter
import warnings;
warnings.filterwarnings('ignore')
from scipy import stats as st
import random
import os
from pathlib import Path
from typing import List, Dict
from google.cloud import bigquery
from mlops_pipelines.helpers.bigquery import get_schema_from_query_job, bq_schema_to_dtypes
from tempfile import TemporaryDirectory
from time import time


from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
#from preprocessing import find_column_type, get_ct_feature_names, get_feature_out, custom_RFE, outlier_function_wrapper, evaluate_preprocessors, Transformer_target_encoder_multiclass, detect_features_high_number_nulls, FakeTransformer
import seaborn as sns


## Loading Dataset from  pickle

In [81]:

df = pd.read_pickle('gs://jorquec_bucket/o2extras/o2extradataset3m.pickle')


In [82]:
df.shape

(36244, 211)

In [83]:
df['target'].value_counts()

0    34259
1     1985
Name: target, dtype: int64

In [84]:
df[['target']].value_counts(normalize=True).reset_index(name='count')

,target,count
0,0,0.945232
1,1,0.054768


In [85]:
pd.set_option("display.max_columns", 1000, "display.max_rows", 1000)

In [86]:
#Function from mlops_pipelines does not include "BIGNUMERIC" so defining it here
def bq_schema_to_dtypes(bq_schema: List[Dict[str, str]]) -> Dict[str, str]:
    mapping_dict = {
        "STRING": "str",
        "INTEGER": "int",
        "FLOAT": "float",
        "NUMERIC": "float",
        "BIGNUMERIC": "float",
        "BOOLEAN": "bool",
    }
    return {column["name"]: mapping_dict.get(column["type"], "object") for column in bq_schema}

In [87]:
# Identify and create a list of column names ending with "_ID"
id_columns = [col for col in df.columns if col.endswith('_ID')]

# Print the list of ID columns
print(id_columns)

[]


In [88]:
# Identify and create a list of column names ending with "_ID"
date_columns = [col for col in df.columns if col.endswith(('_DT','_DATE','_date','_dt'))]

# Print the list of ID columns
print(date_columns)

# use load_date to stratify in the next step

['ref_date', 'contract_start_date', 'load_date', 'tenure_dt']


In [89]:
## Drop date and timestamp columns

df.drop(["ref_date",
         "contract_start_date", 
         "eff_from_timestamp",
         "eff_to_timestamp",
         "tenure_dt",], axis = 1, inplace=True)

In [90]:
df = df.drop('usi', axis=1)

In [91]:
# Drop REPEATED features and others used to identify target( I will use acorn_group)

df.drop(["acorn_type",
         "new_acorn_cat_desc", 
         "perk_type1",
         "perk_type2_later",
         "perk_type2",
         "product_id",
         "product_type_1",
         "product_type_2",
         "product_name",], axis = 1, inplace=True)


In [92]:
# Drop features inputed in feature store and not available in CDM tables
df.drop(["online_competitor_visit_l1m",
         "online_competitor_visit_l3m", "o2_4g_sig_strength_at_PC",
         "device_screen","device_price","sharers_ind","free_video_streaming","second_hand_imei_hd"  ], axis = 1, inplace=True)

In [93]:
# Drop features used to filter base

df = df.drop('CUSTOMER_CAT', axis=1)

In [94]:
print(list(df.columns))

['cycle', 'contract_seq_no_adjusted', 'days_from_ced', 'ooc', 'resign_fanboy_group', 'channel_l1', 'sim_only_ind', 'mins_allowance', 'term', 'tenure_months', 'days_on_current_imei', 'tenure_desc', 'time_on_handset', 'age_id_2', 'familyplan_flag', 'spend_cap_level', 'contract_type', 'refresh', 'flex_refresh_flag', 'first_recent_channel', 'second_recent_channel', 'third_recent_channel', 'total_cs_mins', 'cs_int_flag', 'digital_int_flag', 'wc_int_flag', 'no_of_comms_3m', 'no_of_comms_6m', 'assisted_effort_dur', 'ecn_sent_flag_l1m', 'o2_wifi_days_last_30', 'o2_wifi_days_last_90', 'wifi', 'device_fanboy_group', 'subs', 'offering_subs_rev_inc_vat_amt', 'pac_request_l1m', 'credit_band', 'o2_travel_iz_flag', 'o2_travel_row_flag', 'multiple_mpn_flag', 'os_desc', 'device_os', 'iphone', 'vendor_desc_2', 'is_o25g_ind', 'hs_flag', 'simo_flag', 'total_isrl_browsing_time_l3m', 'isrl_browsing_time_cat', 'domestic_data_costs', 'avg_dom_data_costs', 'ch_dom_data_costs', 'no_of_cells_used', 'avg_tot_ic_c

In [95]:
df.shape

(36244, 187)

In [96]:
def find_column_type(df):
    """
    This function identify categorical, boolean and numerical values.

    Parameters
    ---------
    df : DataFrame
        Usually a DataFrame with training samples that will be used to fit a model.

    Returns
    -------
    categorical_cols : list
        Categorical features.
    bool_cols:
        Boolean features.
    numerical_cols:
        Numerical features.
    """
    
    all_cols=list(df.columns)
    numerical_cols_temp = df.select_dtypes('number').columns
    categorical_cols = list(set(all_cols) - set(numerical_cols_temp))
    bool_cols = [col for col in all_cols if np.isin(df[col].dropna().unique(), [0,1,0.0,1.0]).all()]
    numerical_cols = list(set(numerical_cols_temp) - set(bool_cols))
    
    return categorical_cols,bool_cols,numerical_cols

In [97]:
categorical_cols, bool_cols, numerical_cols = find_column_type(df)

In [98]:
len(numerical_cols)

106

In [99]:
len(categorical_cols)

24

In [100]:
len(bool_cols)

57

### Categorical features

In [101]:
for col in categorical_cols: print(col, len(df[col].dropna().unique()))

utilisation_band 8
acorn_group 19
gender_desc 4
contract_type 4
first_recent_channel 6
age_grp_2 6
third_recent_channel 7
ported_from_grp 7
tv_region_desc_2 15
spend_cap_level 1
second_recent_channel 7
broadband_provider_email 12
os_desc 4
isrl_browsing_time_cat 7
device_fanboy_group 8
channel_l1 6
vendor_desc_2 9
tech_tariff 1
tenure_desc 10
device_os 22
device_group_desc 4
resign_fanboy_group 5
load_date 3
mobile_broadband_ind 2


In [102]:
print(df['device_os'].value_counts(dropna=False))

None                   14951
iOS                    13110
Android                 7960
NONE                      93
Not Known                 56
Nucleus                   22
Not Known,iOS             17
Windows Phone              6
Android Wear               4
S40                        4
Linux                      3
ThreadX                    3
Android,Not Known          3
Android,MTK                2
S30                        2
RTOS                       2
Proprietary OS             1
Android,Window 8           1
Not known                  1
Android,CyanogenMod        1
Not known,iOS              1
KaiOS                      1
Name: device_os, dtype: int64


In [ ]:
#### Group some useless categories, dont do this, as catboost will deal with it.

In [26]:
######df["device_os"].replace(['Not Known,iOS','Not known,iOS','Proprietary OS','RTOS',
 ##       'Windows Phone','Linux','Windows','Nucleus','ThreadX','KaiOS',
 ##       'Windows Mobile','S40','Android Wear','Not Known',
  ##       'S30' ], "Other", inplace=True)

In [103]:
categorical_cols, bool_cols, numerical_cols = find_column_type(df)

In [104]:
categorical_cols.remove('load_date')

In [105]:
#to remove the whitespace from the beginning and at the end of the string
for col in categorical_cols:
    df[col] = df[col].str.strip()


In [106]:
# This is needed because unexpectedly the way the data is read from bigquery using the mlops pipeline functions
# infer null strings as 'None' (string). Reading it directly from bigquery in a notebook you get the expected
# None though, so it must be something to do in the way they read it in the pipeline.
df = df.replace(to_replace=['None'], value = np.nan)

In [107]:
df = df.replace(to_replace=['NONE'], value = np.nan)

In [108]:
for col in categorical_cols:
    print(col)
    print(df[col].value_counts(dropna=False))
    print("----------------------------------------------------------------")

utilisation_band
B.1-25%     18346
A.0          5178
C.26-50%     4806
D.51-80%     2460
NaN          2200
F.100%+      1520
E.81-99%     1317
H. Unkno      417
Name: utilisation_band, dtype: int64
----------------------------------------------------------------
acorn_group
NaN                             9686
1.B  Executive Wealth           3863
3.H  Steady Neighbourhoods      2366
1.C  Mature Money               2283
4.M  Striving Families          2121
2.E  Career Climbers            1802
3.G  Successful Suburbs         1775
4.L  Modest Means               1772
5.P  Struggling Estates         1679
3.J  Starting Out               1416
3.F  Countryside Communities    1372
5.O  Young Hardship             1222
5.Q  Difficult Circumstances    1004
2.D  City Sophisticates          957
4.N  Poorer Pensioners           941
6.R  Not Private Households      619
3.I  Comfortable Seniors         560
1.A  Lavish Lifestyles           460
4.K  Student Life                346
Name: acorn_group, dty

In [109]:
df = df.drop('tech_tariff', axis=1)  # 100% null

In [110]:
df["utilisation_band"].replace(np.nan,"H. Unkno",inplace=True)
df["isrl_browsing_time_cat"].replace(np.nan,"No browsing",inplace=True)
df["broadband_provider_email"].replace(np.nan,"other",inplace=True)
df["device_fanboy_group"].replace(np.nan,"unknown",inplace=True)
df["resign_fanboy_group"].replace(np.nan,"unknown",inplace=True)
df["gender_desc"].replace(np.nan,"Unknown",inplace=True)
df["tenure_desc"].replace(np.nan,"unknown",inplace=True)
df["device_group_desc"].replace(np.nan,"Unknown",inplace=True)
df["os_desc"].replace(np.nan,"Other",inplace=True)
df["age_grp_2"].replace(np.nan,"f. Unknown",inplace=True)
df["channel_l1"].replace(np.nan,"Other",inplace=True)
df["channel_l1"].replace("Null","Other",inplace=True)
df["acorn_group"].replace(np.nan,"Unknown",inplace=True)
df["vendor_desc_2"].replace(np.nan,"OTHER",inplace=True)
##df["new_acorn_cat_desc"].replace(np.nan,"unknown",inplace=True)
df["tv_region_desc_2"].replace(np.nan,"Unknown",inplace=True)
df["first_recent_channel"].replace(np.nan,"Other",inplace=True)
df["first_recent_channel"].replace("Null","Other",inplace=True)
df["second_recent_channel"].replace(np.nan,"Other",inplace=True)
df["second_recent_channel"].replace("Null","Other",inplace=True)
df["third_recent_channel"].replace(np.nan,"Other",inplace=True)
df["third_recent_channel"].replace("Null","Other",inplace=True)
df["spend_cap_level"].replace(np.nan,"unknown",inplace=True)
df["contract_type"].replace(np.nan,"Unknown",inplace=True)
df["ported_from_grp"].replace(np.nan,"Not ported",inplace=True)
df["device_os"].replace(np.nan,"Not Known",inplace=True)
##df["device_model"].replace(np.nan,"Other",inplace=True)

In [111]:
print(list(df.columns))

['cycle', 'contract_seq_no_adjusted', 'days_from_ced', 'ooc', 'resign_fanboy_group', 'channel_l1', 'sim_only_ind', 'mins_allowance', 'term', 'tenure_months', 'days_on_current_imei', 'tenure_desc', 'time_on_handset', 'age_id_2', 'familyplan_flag', 'spend_cap_level', 'contract_type', 'refresh', 'flex_refresh_flag', 'first_recent_channel', 'second_recent_channel', 'third_recent_channel', 'total_cs_mins', 'cs_int_flag', 'digital_int_flag', 'wc_int_flag', 'no_of_comms_3m', 'no_of_comms_6m', 'assisted_effort_dur', 'ecn_sent_flag_l1m', 'o2_wifi_days_last_30', 'o2_wifi_days_last_90', 'wifi', 'device_fanboy_group', 'subs', 'offering_subs_rev_inc_vat_amt', 'pac_request_l1m', 'credit_band', 'o2_travel_iz_flag', 'o2_travel_row_flag', 'multiple_mpn_flag', 'os_desc', 'device_os', 'iphone', 'vendor_desc_2', 'is_o25g_ind', 'hs_flag', 'simo_flag', 'total_isrl_browsing_time_l3m', 'isrl_browsing_time_cat', 'domestic_data_costs', 'avg_dom_data_costs', 'ch_dom_data_costs', 'no_of_cells_used', 'avg_tot_ic_c

In [112]:
## Convert y/n to boolean
df['mobile_broadband_ind'].replace(['Y','N'],[1,0], inplace = True)

In [113]:
#Finding cateorical, boolean and numerical features
categorical_cols, bool_cols, numerical_cols = find_column_type(df)

In [114]:
#Lower case
df.columns = [x.lower() for x in df.columns]

### Numerical features 

In [115]:
#Null values

nans = df.isna().sum()/df.shape[0] *100
nans[nans>0].sort_values(ascending=False)

avg_standard_c                    100.000000
avg_standard_d                    100.000000
max_snacking_bought_l3m           100.000000
ch_ic_mob_wkd_r                   100.000000
interconnect_mob_day_r            100.000000
avg_ic_mob_wkd_r                  100.000000
avg_ic_mob_eve_r                  100.000000
avg_ic_mob_day_r                  100.000000
ncx_days_20to40_l3m               100.000000
interconnect_revenue              100.000000
avg_other_d                       100.000000
premium_costs                     100.000000
ch_standard_d                     100.000000
ch_ic_natl_r                       99.986205
avg_ic_natl_r                      99.986205
per_standard_d                     97.922415
pac_requested_l12m                 97.194018
pac_request_l1m                    97.194018
o2_travel_row_flag                 95.858625
total_sms_1st_5days                95.265423
total_voice_count_1st_5days        94.981238
total_voice_duration_1st_5days     94.981238
total_data

In [116]:
# Identify and create a list of features with many Nulls - just check 

majoritynull = [col for col in df.columns if (df[col].isna().sum()/ df.shape[0] *100) > 99.0]                                          

In [117]:
majoritynull

['avg_standard_d',
 'avg_standard_c',
 'ch_standard_d',
 'premium_costs',
 'avg_other_d',
 'interconnect_revenue',
 'avg_ic_mob_day_r',
 'avg_ic_natl_r',
 'avg_ic_mob_eve_r',
 'avg_ic_mob_wkd_r',
 'interconnect_mob_day_r',
 'ch_ic_natl_r',
 'ch_ic_mob_wkd_r',
 'max_snacking_bought_l3m',
 'ncx_days_20to40_l3m']

In [118]:
# Remove high null columns   I will keep it  as it could be only data deffect?
#df = df.drop(columns=majoritynull)

In [119]:
##  Checks Unique Values and  Variance

In [120]:
cols_to_drop = [col for col in df.columns if len(df[col].unique()) == 1]
cols_to_drop

['ooc',
 'spend_cap_level',
 'refresh',
 'flex_refresh_flag',
 'avg_standard_d',
 'avg_standard_c',
 'ch_standard_d',
 'premium_costs',
 'avg_other_d',
 'interconnect_revenue',
 'avg_ic_mob_day_r',
 'avg_ic_mob_eve_r',
 'avg_ic_mob_wkd_r',
 'interconnect_mob_day_r',
 'ch_ic_mob_wkd_r',
 'additional_data_allowance_1mth',
 'additional_data_allowance_2mth',
 'additional_data_allowance_3mth',
 'max_snacking_bought_l3m',
 'early_life',
 'ncx_days_20to40_l3m']

In [121]:
# Now looking at & dropping other colums with low variance.

In [122]:
pd.set_option('display.max_rows', None)
threshold = 0.05
df.std()[df.std() < threshold]

ooc                               0.000000
familyplan_flag                   0.000000
refresh                           0.000000
flex_refresh_flag                 0.000000
total_cs_mins                     0.000000
cs_int_flag                       0.005253
assisted_effort_dur               0.000000
ecn_sent_flag_l1m                 0.027286
o2_wifi_days_last_30              0.000000
o2_wifi_days_last_90              0.000000
o2_travel_iz_flag                 0.000000
o2_travel_row_flag                0.000000
multiple_mpn_flag                 0.000000
is_o25g_ind                       0.000000
hs_flag                           0.005429
simo_flag                         0.000000
c2m_ind                           0.000000
per_standard_d                    0.000000
standard_d                        0.000000
standard_c                        0.000000
myo2                              0.000000
my_o2_days_access_last_30         0.000000
my_o2_days_access_last_90         0.000000
online_upgr

In [47]:
# drop some that contain only nulls

In [123]:
cols_to_drop = ['familyplan_flag',
                'total_cs_mins',
                'assisted_effort_dur',
                'o2_wifi_days_last_30',
                'o2_wifi_days_last_90',
                'o2_travel_iz_flag',
                'c2m_ind',
                'per_standard_d',
                'standard_d',
                'myo2',
                'my_o2_days_access_last_30',
                'my_o2_days_access_last_90',
                'additional_data_allowance_1mth',
                'additional_data_allowance_2mth',
                'additional_data_allowance_3mth',
                'standard_c',
                'is_o25g_ind',
                'o2_travel_row_flag',
                'max_snacking_bought_l3m'
                ]
df=df.drop(cols_to_drop,axis=1)

In [124]:
cols_to_drop2 = ['avg_standard_d',
                 'avg_standard_c',
                 'ch_standard_d',
                 'premium_costs',
                 'avg_other_d',
                 'online_upgrade_visit_l1m',
                 'interconnect_revenue',
                 'avg_ic_mob_day_r',
                 'avg_ic_mob_eve_r',
                 'avg_ic_mob_wkd_r',
                 'interconnect_mob_day_r',
                 'ch_ic_mob_wkd_r',
                 'ncx_days_20to40_l3m'
                ]
df=df.drop(cols_to_drop2,axis=1)

In [125]:
df.shape

(36244, 154)

In [126]:
categorical_cols, bool_cols, numerical_cols = find_column_type(df)

In [127]:
categorical_cols.remove('load_date')

### Boolean features

In [128]:
bool_cols

['ooc',
 'sim_only_ind',
 'refresh',
 'flex_refresh_flag',
 'cs_int_flag',
 'digital_int_flag',
 'wc_int_flag',
 'ecn_sent_flag_l1m',
 'wifi',
 'pac_request_l1m',
 'multiple_mpn_flag',
 'iphone',
 'hs_flag',
 'simo_flag',
 'retail_int_flag',
 'mobile_broadband_ind',
 'plan_cca_ind',
 'insurance',
 'opted_in_flag',
 'ported_flag',
 'arrears',
 'pac_requested_l12m',
 'early_life',
 'bt_overbuild',
 'alt_net_overbuild',
 'target']

In [129]:
nans = df[bool_cols].isna().sum()/df.shape[0] *100
nans[nans>0]

cs_int_flag            0.013795
digital_int_flag       0.013795
wc_int_flag            0.013795
ecn_sent_flag_l1m      0.013795
wifi                   0.013795
pac_request_l1m       97.194018
multiple_mpn_flag     13.461538
iphone                41.250966
hs_flag                6.381746
simo_flag              6.381746
retail_int_flag        0.013795
pac_requested_l12m    97.194018
bt_overbuild          27.149321
alt_net_overbuild     27.149321
dtype: float64

In [130]:
for col in bool_cols:
    print(col)
    print(df[col].value_counts(dropna=False))
    print("----------------------------------------------------------------")

ooc
0    36244
Name: ooc, dtype: int64
----------------------------------------------------------------
sim_only_ind
1    23776
0    12468
Name: sim_only_ind, dtype: int64
----------------------------------------------------------------
refresh
0    36244
Name: refresh, dtype: int64
----------------------------------------------------------------
flex_refresh_flag
0    36244
Name: flex_refresh_flag, dtype: int64
----------------------------------------------------------------
cs_int_flag
0.0    36238
NaN        5
1.0        1
Name: cs_int_flag, dtype: int64
----------------------------------------------------------------
digital_int_flag
0.0    18613
1.0    17626
NaN        5
Name: digital_int_flag, dtype: int64
----------------------------------------------------------------
wc_int_flag
0.0    34496
1.0     1743
NaN        5
Name: wc_int_flag, dtype: int64
----------------------------------------------------------------
ecn_sent_flag_l1m
0.0    36212
1.0       27
NaN        5
Name: ec

In [131]:
# Check Correlation--

In [56]:
# Create correlation matrix
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df[numerical_cols].columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df[numerical_cols].corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df[numerical_cols])
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(df, 40))

Top Absolute Correlations
total_isrl_browsing_time_l3m            j7_total_duration_90                      1.000000
extra_interactionbrow_duration_last180  extra_interactionbrow_duration_last90     1.000000
offering_subs_rev_inc_vat_amt           subs                                      1.000000
data_util                               utilisation                               1.000000
term                                    contract_month                            1.000000
extra_interactionbrow_duration_last180  extra_interactionbrow_duration_last150    1.000000
extra_interactionbrow_duration_last150  extra_interactionbrow_duration_last90     1.000000
extra_interactionbrow_duration_last180  extra_interactionbrow_duration_last120    1.000000
extra_interactionbrow_duration_last30   extra_interactionbrow_duration_last60     1.000000
extra_interactionbrow_duration_last120  extra_interactionbrow_duration_last90     1.000000
cycle                                   contract_seq_no_adjusted

In [132]:
# drop a subset 

corrs=['contract_seq_no_adjusted','j7_total_duration_90','utilisation','offering_subs_rev_inc_vat_amt',
        'contract_month','days_on_current_imei','voice_per']

In [133]:
df.shape

(36244, 154)

In [134]:
# As intending to use CatGBoost not very concerned about correlated variables,
#deleted only 100% correlated

In [135]:
df = df.drop(columns=corrs)

In [136]:
df.shape

(36244, 147)

In [137]:
print(list(df.columns))

['cycle', 'days_from_ced', 'ooc', 'resign_fanboy_group', 'channel_l1', 'sim_only_ind', 'mins_allowance', 'term', 'tenure_months', 'tenure_desc', 'time_on_handset', 'age_id_2', 'spend_cap_level', 'contract_type', 'refresh', 'flex_refresh_flag', 'first_recent_channel', 'second_recent_channel', 'third_recent_channel', 'cs_int_flag', 'digital_int_flag', 'wc_int_flag', 'no_of_comms_3m', 'no_of_comms_6m', 'ecn_sent_flag_l1m', 'wifi', 'device_fanboy_group', 'subs', 'pac_request_l1m', 'credit_band', 'multiple_mpn_flag', 'os_desc', 'device_os', 'iphone', 'vendor_desc_2', 'hs_flag', 'simo_flag', 'total_isrl_browsing_time_l3m', 'isrl_browsing_time_cat', 'domestic_data_costs', 'avg_dom_data_costs', 'ch_dom_data_costs', 'no_of_cells_used', 'avg_tot_ic_cost', 'ch_tot_ic_cost', 'total_interconnect_cost', 'arpu', 'airtime_margin', 'avg_arpu', 'avg_airtime_margin', 'ch_arpu', 'ch_airtime_margin', 'revenue_amount', 'ch_billed_revenue', 'billed_revenue', 'avg_billed_revenue', 'domestic_data_usage', 'tota

In [138]:
df_clean=df

In [139]:
df_clean.shape

(36244, 147)

In [140]:
## To save in my GCP bucket : 
df.to_pickle('gs://jorquec_bucket/o2extras/o2extradataset_preprocessed.pickle')

In [2]:
## To load it back again: 
df = pd.read_pickle('gs://jorquec_bucket/o2extras/o2extradataset_preprocessed.pickle')

In [ ]:
#For null values, I will leave them in rather than filling with a median as CatGboost has 
#the capability to handle null values

In [141]:
df.shape

(36244, 147)

In [142]:
categorical_cols, bool_cols, numerical_cols = find_column_type(df)

In [143]:
pd.set_option('display.max_rows', 500)

In [144]:
num_medians = df[numerical_cols].median()
num_medians

ncx_days_60to80_l3m                          53.000000
ch_dom_data_usg                               2.360402
avg_onnet_call_c                             29.000000
data_util                                     0.092600
extra_interactionbrow_duration_last180        0.000000
avg_dom_data_costs                          151.616753
ch_billed_revenue                             2.513380
extra_interactionbrow_duration_last30         0.000000
cycle                                         3.000000
avg_tot_ic_cost                              -3.680100
ncx_score                                     9.000000
roaming_cost                                  0.000000
ch_offnet_call_d                              2.503490
time_on_handset                              19.000000
extra_interactionbrow_duration_last150        0.000000
avg_offnet_call_d                           320.597500
term                                         24.000000
avg_airtime_margin                          113.542300
age_id_2  

In [145]:
bools_med = df[bool_cols].median()
bools_med

ooc                     0.0
sim_only_ind            1.0
refresh                 0.0
flex_refresh_flag       0.0
cs_int_flag             0.0
digital_int_flag        0.0
wc_int_flag             0.0
ecn_sent_flag_l1m       0.0
wifi                    0.0
pac_request_l1m         1.0
multiple_mpn_flag       0.0
iphone                  1.0
hs_flag                 0.0
simo_flag               0.0
retail_int_flag         0.0
mobile_broadband_ind    0.0
plan_cca_ind            0.0
insurance               0.0
opted_in_flag           0.0
ported_flag             0.0
arrears                 0.0
pac_requested_l12m      1.0
early_life              1.0
bt_overbuild            0.0
alt_net_overbuild       0.0
target                  0.0
dtype: float64

In [146]:
bools_mode = df[bool_cols].mode()
bools_mode

,ooc,sim_only_ind,refresh,flex_refresh_flag,cs_int_flag,digital_int_flag,wc_int_flag,ecn_sent_flag_l1m,wifi,pac_request_l1m,multiple_mpn_flag,iphone,hs_flag,simo_flag,retail_int_flag,mobile_broadband_ind,plan_cca_ind,insurance,opted_in_flag,ported_flag,arrears,pac_requested_l12m,early_life,bt_overbuild,alt_net_overbuild,target
0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,1.0,1,0.0,0.0,0


In [147]:
# to save medians
with open('/home/jupyter/o2extras/datasets/num_medians.pkl', 'wb') as f: 
    pickle.dump(num_medians, f)

In [ ]:
# to read later
with open('/home/jupyter/o2extras/datasets/num_medians.pkl', 'rb') as f: 
    num_medians = pickle.load(f)

In [148]:
# to save medians for Bools  -- same as mode
with open('/home/jupyter/o2extras/datasets/bools_medians.pkl', 'wb') as g: 
    pickle.dump(bools_med, g)

In [ ]:
# to read later Bolls median 
with open('/home/jupyter/o2extras/datasets/bools_medians.pkl', 'rb') as g: 
    bool_med = pickle.load(g)

In [149]:
#Box Plot

In [150]:
## Checks

In [151]:

grouped = df.groupby(['target'])

split = grouped.mean()

In [152]:
split

,cycle,days_from_ced,ooc,sim_only_ind,mins_allowance,term,tenure_months,time_on_handset,age_id_2,refresh,flex_refresh_flag,cs_int_flag,digital_int_flag,wc_int_flag,no_of_comms_3m,no_of_comms_6m,ecn_sent_flag_l1m,wifi,subs,pac_request_l1m,credit_band,multiple_mpn_flag,iphone,hs_flag,simo_flag,total_isrl_browsing_time_l3m,domestic_data_costs,avg_dom_data_costs,ch_dom_data_costs,no_of_cells_used,avg_tot_ic_cost,ch_tot_ic_cost,total_interconnect_cost,arpu,airtime_margin,avg_arpu,avg_airtime_margin,ch_arpu,ch_airtime_margin,revenue_amount,ch_billed_revenue,billed_revenue,avg_billed_revenue,domestic_data_usage,total_voice_mins,total_voice_calls,avg_dom_data_usg,off_net_call_c,avg_offnet_call_c,avg_offnet_call_d,roam_data_usg,avg_roam_data_usg,avg_onnet_call_c,avg_onnet_call_d,sms_per,per_offnet_call_c,per_offnet_call_d,per_onnet_call_d,ch_dom_data_usg,ch_offnet_call_d,ch_onnet_call_d,ch_sms_c,per_sms_c,data_util,roaming_cost,avg_sms_c,off_net_call_d,on_net_call_c,on_net_call_d,sms_c,retail_int_flag,avg_ic_natl_r,avg_tot_ic_rev,interconnect_natl_r,avg_ic_sms_r,interconnect_sms_r,ch_ic_sms_r,ch_ic_natl_r,total_interconnect_rev,ch_tot_ic_rev,mobile_broadband_ind,data_uk_allow_mb_qty,plan_cca_ind,in_bundle_data_usage_1mth,in_bundle_data_usage_2mth,in_bundle_data_usage_3mth,handset_price,insurance,opted_in_flag,ported_flag,simo_subscr_ids,handset_subscr_ids,total_sms_1st_5days,total_voice_count_1st_5days,total_voice_duration_1st_5days,total_data_usage_1st_5days,total_subscr_ids,no_of_current_connections,arrears,j6_total_duration_90,total_voice_count,domestic_voice_duration,comms_engaged_with,comms_delivered,total_assist_duration_90,pac_requested_l12m,main_data_bolton_allowance,early_life,bt_overbuild,alt_net_overbuild,extra_interactionjourney_30firstdays,extra_interactionbrow_duration_last30,extra_interactionbrow_duration_last60,extra_interactionbrow_duration_last90,extra_interactionbrow_duration_last120,extra_interactionbrow_duration_last150,extra_interactionbrow_duration_last180,ave_ncx_l3m,ncx_days_0to20_l3m,ncx_days_40to60_l3m,ncx_days_60to80_l3m,ncx_days_80to100_l3m,ncx_score,final_data_allowance
target,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3.664320,-723.559415,0.0,0.656324,5.992100e+06,24.266499,69.951925,22.631550,48.752643,0.0,0.0,0.000029,0.470456,0.046068,10.980177,11.747562,0.000730,0.254160,19.117252,0.934737,6.216312,0.0,0.611464,0.000031,0.0,42438.429025,212.105601,170.257147,1.405586,172.635608,-8.418000,1.284365,-10.100627,265.056029,327.712148,136.292564,142.963497,2.283941,2.472566,338.208896,2.104248,324.329575,171.122267,80528.431227,2081.984478,707.048159,45840.440170,532.613385,294.249676,773.098483,3929.368486,4659.972619,88.529620,299.204888,24.092372,53.921691,65.438550,16.959701,2.016603,2.094692,2.206253,2.156142,21.230649,0.216666,-2.234767,151.832024,1452.386370,151.907858,522.748668,283.303180,0.008291,0.05,8.060875,9.494874,3.690805,4.302397,1.167363,267.835917,9.692566,1.277541,0.001635,7.450389e+06,0.343589,5429.497379,4016.892983,2728.003023,28.412458,0.019936,0.253860,0.061823,0.671806,0.367536,36.581938,8.749296,25.680682,1180.992455,1.058590,1.018231,0.016025,5.870007,46.433247,163.421264,0.287587,1.568722,4556.452592,1.0,3.613202e+07,1.0,0.295452,0.197832,0.004729,0.011413,0.011413,0.013252,0.013252,0.013252,0.013252,5.010206,79.577946,25.245721,56.397000,101.758510,6.624365,7.149920e+06
1,5.097881,-748.609068,0.0,0.650378,4.495450e+04,25.085642,96.352141,25.699931,41.351782,0.0,0.0,0.000000,0.761209,0.083123,18.726952,19.822670,0.001008,0.450378,23.832363,0.895522,7.205882,0.0,0.637248,0.000000,0.0,55944.890065,246.502295,195.083777,1.426500,215.019211,-7.792909,1.348735,-9.572367,281.717185,272.011608,114.056203,120.614337,2.625475,2.549634,379.274298,2.533024,368.913026,158.248726,138162.573111,2262.570202,757.211800,84882.584977,564.927383,284.523230,764.493021,6402.256652,5890.300456,87.766492,299.494921,23.9

In [153]:
df.subs.describe().apply("{0:.8f}".format)

count    36244.00000000
mean        19.37548753
std          8.07942818
min          0.00000000
25%         12.00000000
50%         18.00000000
75%         25.00000000
max         60.00000000
Name: subs, dtype: object

In [154]:
df.age_id_2.describe().apply("{0:.8f}".format)

count    26477.00000000
mean        48.38395589
std         16.09091003
min         18.00000000
25%         35.00000000
50%         46.00000000
75%         60.00000000
max        123.00000000
Name: age_id_2, dtype: object

In [155]:
df.handset_price.describe().apply("{0:.8f}".format)

count    11757.00000000
mean        28.57447903
std        116.55626581
min          0.00000000
25%          0.00000000
50%          0.00000000
75%          0.00000000
max       1769.00000000
Name: handset_price, dtype: object

In [156]:
df.final_data_allowance.describe().apply("{0:.8f}".format)

count        36041.00000000
mean       7031333.48392109
std       26884314.32594362
min            250.00000000
25%           6144.00000000
50%          25600.00000000
75%         122880.00000000
max      999999999.00000000
Name: final_data_allowance, dtype: object